#### 2-1. Nate 뉴스기사 제목 스크래핑하기
https://news.nate.com/recent?mid=n0100
최신뉴스, 정치 , 경제, 사회, 세계, IT/과학 
6개의 섹션의 뉴스를 출력하는 함수를 생성하여 스크래핑 하기

Image, 기사제목, 기사링크


□ 뉴스기사의 Image를 출력 하세요 
1) Image의 도메인이름이 포함된 url과 src 속성의 img 경로를 합치려면 urljoin 함수를 사용하세요.
    from urllib.parse import urljoin
    
    url = ‘https://news.nate.com/recent?mid=n0100’
    src=’ //thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/na/2025/07/23/7408335_high.jpg’

2) Image 출력은 Image 클래스와 display 함수를 사용하세요.
    from IPython.display import Image, display

3) img 엘리먼트의 존재 여부를 체크하신 후에 src 속성의 이미지를 경로를 추출하기
  => Image 가 없는 뉴스도 있기 때문에


In [ ]:
import requests
import os
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display
import datetime

section_dict = {'최신뉴스':'n0100','정치':'n0200','경제':'n0300','사회':'n0400','세계':'n0500','IT/과학':'n0600'}

def print_news(section_name):
    if len(section_name) >= 2: #입력한 받은 문자의 길이가 2이상일 경우에만 변환
        for i in range(0,len(section_dict)): #Section_Dict의 길이 만큼 반복
            if section_name in list(section_dict.keys())[i]: #입력한 문자가 DICT에 포함되어 있는지 확인
                section_name = list(section_dict.keys())[i] # 입력한 문자가 DICT에 포함되어 있는 문자라면 DICT의 키 값으로 변환
            else:
                pass
    else:
        section_name = "최신뉴스" #기본 값

    sid = section_dict.get(section_name,"최신뉴스")
    url = f'https://news.nate.com/section?mid={sid}'
    print(f'{section_name} 뉴스 {url}')
    req_header = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.3)'
    }
    res = requests.get(url, headers=req_header)
    if res.ok:
        soup = BeautifulSoup(res.text,'html.parser')
        for a_tag, img_tag in zip(soup.select("a[href*='//news.nate.com/view/']"), soup.select('img')):
            if a_tag['href'][-5:]==sid:
                if '\t' in a_tag.text.strip(): #기사의 부제목 유무 확인
                    title = a_tag.text.strip().split('\t')[0] #부제목을 제외한 기사의 제목만 title변수에 저장
                else:
                    title = a_tag.text.strip()#부제목이 없다면 그대로 title에 저장
                    
                link = "https:" + a_tag['href']
                if "jpg" in str(a_tag) or "RMG0" in str(a_tag): #a_tagdp RMG문자열이 포함되어 있다면 이미지 존재
                    src = img_tag['src']
                    url_join = urljoin(base=url, url=src, allow_fragments=True)
                    res_img = requests.get(url_join, headers=req_header)
                    img_data=res_img.content
                    now = str(datetime.datetime.now().strftime('%S')) #파일명 초단위 이름 지정
                    file_name = "test_"+now+".jpg" #파일 실행 경로 이미지 파일 저장
                    with open(file_name,'wb') as file:
                        file.write(img_data)

                    p_img = display(Image(filename=file_name, width=300)) #이미지 파일 변수 지정
                else:
                    p_img = "이미지 없음"
                print(p_img, title, link)
            else:
                pass
    else:
        print(f'Error Code={res.status_code}')

print_news('IT')


#### 2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기
 하나의 웹툰의 제목(title)과 회차번호(no),회차의URL(url) 을 입력으로 받는 함수를 선언합니다
  ex) def download_one_episode(title,no,url):

In [131]:
import requests
from bs4 import BeautifulSoup
import os

#테스트 정보
#test_title = '육아의 계절'
#test_no = = '229'
#test_url = 'https://comic.naver.com/bestChallenge/detail?titleId=742835&no=234'

def download_one_episode(title,no,url):
    webtoon_url = url
    req_header = {
        'referer':webtoon_url,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.3'
    }
    res = requests.get(webtoon_url, headers=req_header)
    if res.ok:
        soup = BeautifulSoup(res.text,'html.parser')
        img_tags = soup.select("img[id*='content_image_0']")
        img_url_list = [img_tag['src'] for img_tag in img_tags]

        for img_url in img_url_list:
            res = requests.get(img_url, headers=req_header)
            img_data=res.content
            file_name = "test.jpg"
            with open(file_name,'wb') as file:
                file.write(img_data)
    else:
        print(f'Error Code={res.status_code}')

title_input = input("웹툰의 제목을 입력해 주세요: ")
no_input = input("웹툰의 회차를 입력해 주세요: ")
url_input = input("웹툰의 URL을 입력해 주세요: ")

download_one_episode(title_input, no_input, url_input)

#### 2-3. 하나의 네이버 웹툰과 여러개의 회차에 대한 Image 다운로드 하기
하나의 웹툰의 제목(title)과 회차를 알 수 있는 url을 입력으로 받는 함수를 선언합니다. ex) def download_all_episode(title, episode_url):